<a href="https://colab.research.google.com/github/alexisakov/TinkoffCOVIDIndex/blob/master/TinkoffCOVIDIndexScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chose file path for saved data:

In [0]:
filepath = r'C:\Users\YOURUSERNAME\Downloads\TinkoffCOVIDIndex.xlsx'

In [0]:
import json, pandas as pd, requests

In [0]:
tall = json.loads(requests.get(f'https://index.tinkoff.ru/corona-index/papi/charts?region=all').content)

In [0]:
# process 'total'
ttlnames = ['Index','ConsumerActivity','OnlinePayShare']

startdate = tall['total']['start']
startdate = startdate[6:] + '-' + startdate[3:5] + '-' + startdate[0:2]

dtrange = pd.date_range(startdate, periods=len(tall['total']['points']), freq='D')
dftotal = pd.DataFrame(tall['total']['points'], index=dtrange, columns=ttlnames)

In [0]:
# process 'categories'

dictcat = []

for c in tall['categories']:
    if tall['categories'][c] == {}:
        continue
    startdate = tall['categories'][c]['start']
    startdate = startdate[6:] + '-' + startdate[3:5] + '-' + startdate[0:2]
    dtrange = pd.date_range(startdate, periods=len(tall['categories'][c]['points']), freq='D')
    dictcat.append(pd.DataFrame(tall['categories'][c]['points'], index=dtrange, columns=[c]))
#    print(c)
dfcategories = pd.concat(dictcat, axis=1)

In [0]:
# process businessTotal
ttlnames = ['Turnoveer']

startdate = tall['businessTotal']['start']
startdate = startdate[6:] + '-' + startdate[3:5] + '-' + startdate[0:2]
dtrange = pd.date_range(startdate, periods=len(tall['businessTotal']['points']), freq='W')

dfBusinessTotal = pd.DataFrame(tall['businessTotal']['points'], index=dtrange, columns=ttlnames)

In [0]:
# process 'categories'
dictcat = []

for c in tall['businessCategories']:
    if tall['businessCategories'][c] == {}:
        continue
    startdate = tall['businessCategories'][c]['start']
    startdate = startdate[6:] + '-' + startdate[3:5] + '-' + startdate[0:2]
    dtrange = pd.date_range(startdate, periods=len(tall['businessCategories'][c]['points']), freq='W')
    dictcat.append(pd.DataFrame(tall['businessCategories'][c]['points'], index=dtrange, columns=[c]))
#    print(c)
dfBusinessCategories = pd.concat(dictcat, axis=1)

Save dataframes to xl using this recipe: https://stackoverflow.com/questions/14225676/save-list-of-dataframes-to-multisheet-excel-spreadsheet


In [0]:
def save_xls(list_dfs, xls_path):
    import pandas as pd
    with pd.ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()

In [0]:
save_xls([dftotal,dfcategories,dfBusinessTotal,dfBusinessCategories],filepath)